<a href="https://colab.research.google.com/github/Sidstark12/Soc-intro-to-Machine-Intelligence/blob/SOC-Q-Learning/SoC_final_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scenario - Slippery lake Environment

Build an AI agent from scratch, capable of traversing a predetermined environment, despite the inherent stochasticity.
The final goal is to maximise our chances of reaching the final goal in the minimum number fo steps witout touhcing any pitfalls intiating from the white tiles.
We will use Q-Learning to accomplish this task.

In [ ]:
import numpy as np
import random

## Defining Environment

### States
The states in the environment are all of the possible places we can move through, these are (*grey squares*) pit falls, moving tiles (*white squares*),etc.The final goal is the *green square*.

* Grey and green are the terminal states.
* Orange squares are super slippery surface.
* Blue squares are non-slippery surface.
* White squares are slightly slippery surface.

In [ ]:
#define the shape of the environment (i.e., its states)
envi_rows = 7
envi_columns = 7

# A 3D array is created to take in account of environment as rows and columns, actions of the agent.
#"Action" dimension consists of 4 layers
#The value of each (state, action) pair is initialized to 0.
q_values = np.zeros((envi_rows, envi_columns, 4))

### Actions
The actions availabel for the AI agent are to move in the four directions:
* North
* East
* West
* South

In [ ]:
#numeric action codes:0 = North, 1 = East, 2 = West, 3 = South.
actions = ['North','East','West','South']

### Rewards
Negative rewards (i.e., punishments) are used for all states except the goal.

In [ ]:
# Create a 2D numpy array to hold the reward for each state.
rewards = np.full((envi_rows,envi_columns),-1.)
rewards[3,3] = 100

#value for non-slippery surface = -25
#value for super slippery surface = -50
#now defining the aile values
aisle = {} #locations are being stored in dictionary
aisle[0] = [3]
aisle[1] = [1,5]
aisle[2] = []
aisle[3] = [2,4,6]
aisle[4] = [3]
aisle[5] = [1]
aisle[6] = [5]

#set the rewards for the aisle locations (grey squares)
for row_index in range(0,7):
    for column_index in aisle[row_index]:
        rewards[row_index,column_index] = -100

# now giving the values of blue sqaures
aisle_blue = {}
aisle_blue[0]= []
aisle_blue[1]=[2,3,4]
aisle_blue[2]=[]
aisle_blue[3]=[]
aisle_blue[4]=[2]
aisle_blue[5]=[4]
aisle_blue[6]=[0]

for row_index in range(1,7):
    for column_index in aisle_blue[row_index]:
        rewards[row_index,column_index] = 50
        
# now giving values to orange sqaures
aisle_orange = {}
aisle_orange[0]=[]
aisle_orange[1]=[]
aisle_orange[2]=[0]
aisle_orange[3]=[]
aisle_orange[4]=[]
aisle_orange[5]=[5]
aisle_orange[6]=[]

for row_index in range(1,7):
    for column_index in aisle_orange[row_index]:
        rewards[row_index,column_index] = -50
#print the matrix
for row in rewards:
    print(row)

[  -1.   -1.   -1. -100.   -1.   -1.   -1.]
[  -1. -100.   50.   50.   50. -100.   -1.]
[-50.  -1.  -1.  -1.  -1.  -1.  -1.]
[  -1.   -1. -100.  100. -100.   -1. -100.]
[  -1.   -1.   50. -100.   -1.   -1.   -1.]
[  -1. -100.   -1.   -1.   50.  -50.   -1.]
[  50.   -1.   -1.   -1.   -1. -100.   -1.]


Now we have our envieronment ready, hence we'll proceed to making of helper functions for our agent. Some points that we need to attend are
* According to the given condtion the intial postion of starting should be a white sqaure.
* The actions of the agent will be selected using the *epsilon greedy algorithm*

In [ ]:
# Now we define a function that will determine if the location selected is terminal state or not
# we'll determine the position on the basis of reward given to it

# this functions determines if the surface is a terminal state or not
def argmax(q_values):
    top = float("-inf")
    ties = []
    for i in range(len(q_values)):
        if q_values[i] > top:
            top, ties = q_values[i], [i]
        elif q_values[i] == top:
            ties.append(i)
    ind = np.random.choice(ties)
    return ind


def terminal_state(current_row_index,current_column_index):
    if rewards[current_row_index,current_column_index] == -100 or rewards[current_row_index,current_column_index] == 100:
        return True
    else:
        return False
# till now we are able to determine if the surface is a terminal state or not

# now select a random, non terminal starting state
def get_starting_location():
    current_row_index = np.random.randint(envi_rows)
    current_column_index = np.random.randint(envi_columns)
    while terminal_state(current_row_index,current_column_index):
        current_row_index = np.random.randint(envi_rows)
        current_column_index = np.random.randint(envi_columns)
    return current_row_index, current_column_index

# till now our random white tile starting surface is selected 

# now we define epsilon greedy algorithm to determine the action the agent needs to take

def get_next_action(current_row_index,current_column_index):
    action_index = argmax(q_values[current_row_index, current_column_index])
    return action_index

    

# this fucntion will get the next location 
def get_next_location(current_row_index,current_column_index,action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'North' and current_row_index >0:
        new_row_index -= 1
    elif actions[action_index] == 'East' and current_column_index < envi_columns - 1:
        new_column_index += 1
    elif actions[action_index] == 'West' and current_column_index >0:
        new_column_index -= 1 
    elif actions[action_index] == 'South' and current_row_index < envi_rows - 1:
        new_row_index += 1
    return new_row_index, new_column_index  

In [ ]:
# training parameters
alpha = 0.1 # learning rate
gamma = 0.99
epsilon = 0.1

# run through 15500 episodes
for episode in range(100001):
    row_index, column_index = get_starting_location()
    while not terminal_state(row_index, column_index):
        if random.uniform(0, 1) < epsilon:
            action_index = np.random.choice([0,1,2,3]) 
        else:
            action_index = get_next_action(row_index, column_index)
        
        new_row_index, new_column_index = get_next_location(row_index,column_index,action_index)
        reward_new = rewards[new_row_index, new_column_index]
        
        old_value = q_values[row_index,column_index,action_index]
        next_max = np.max(q_values[new_row_index,new_column_index])
        
        new_value = (1 - alpha) * old_value + alpha * (reward_new + gamma * next_max)
        q_values[row_index,column_index,action_index] = new_value
        
        row_index = new_row_index
        column_index = new_column_index
    if episode % 1000 == 0:
        print(f"Episode: {episode}")        
print('training complete')

Episode: 0
Episode: 1000
Episode: 2000
Episode: 3000
Episode: 4000
Episode: 5000
Episode: 6000
Episode: 7000
Episode: 8000
Episode: 9000
Episode: 10000
Episode: 11000
Episode: 12000
Episode: 13000
Episode: 14000
Episode: 15000
Episode: 16000
Episode: 17000
Episode: 18000
Episode: 19000
Episode: 20000
Episode: 21000
Episode: 22000
Episode: 23000
Episode: 24000
Episode: 25000
Episode: 26000
Episode: 27000
Episode: 28000
Episode: 29000
Episode: 30000
Episode: 31000
Episode: 32000
Episode: 33000
Episode: 34000
Episode: 35000
Episode: 36000
Episode: 37000
Episode: 38000
Episode: 39000
Episode: 40000
Episode: 41000
Episode: 42000
Episode: 43000
Episode: 44000
Episode: 45000
Episode: 46000
Episode: 47000
Episode: 48000
Episode: 49000
Episode: 50000
Episode: 51000
Episode: 52000
Episode: 53000
Episode: 54000
Episode: 55000
Episode: 56000
Episode: 57000
Episode: 58000
Episode: 59000
Episode: 60000
Episode: 61000
Episode: 62000
Episode: 63000
Episode: 64000
Episode: 65000
Episode: 66000
Episode:

In [ ]:
# this function will select the shortest path from the current location to the final position
# it help in moving the agent
def get_shortest_path(start_row_index,start_column_index):
    shortest_path = []
    i = 0
    if terminal_state(start_row_index,start_column_index):
        print ("terminal state")
    else:
        current_row_index,current_column_index = start_row_index,start_column_index
        while not terminal_state(current_row_index,current_column_index):
            i = i+1
            if i>100:
                break
            action_index = get_next_action(current_row_index, current_column_index)
            current_row_index,current_column_index = get_next_location(current_row_index,current_column_index,action_index)
            shortest_path.append(actions[action_index])
    return shortest_path   
                

In [ ]:
start_row_index, start_column_index = get_starting_location()
# start_row_index, start_column_index = 1,4
print(f"start state {start_row_index}, {start_column_index}")
shortest_path = get_shortest_path(start_row_index,start_column_index)
shortest_path

start state 0, 5


['West',
 'South',
 'West',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'West',
 'East',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'West',
 'East',
 'West',
 'East',
 'East',
 'West',
 'West',
 'East',
 'East',
 'West',
 'East',
 'West',
 'East']